# 01. Install dependancies

In [4]:
!pip install reportlab
!pip install numpy
!pip install pandas

In [6]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from reportlab.pdfgen import canvas
from io import BytesIO

# Load the dataset of resumes
dataset = pd.read_csv('UpdatedResumeDataSet.csv')

# Split the dataset into features and targets
features = dataset['Category']
targets = dataset['Resume']

# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Transform the features into vectors
features_vectors = vectorizer.fit_transform(features)

# Create a logistic regression classifier
classifier = LogisticRegression()

# Train the classifier
classifier.fit(features_vectors, targets)

# Get user details
name = input("Enter your name: ")
email = input("Enter your email: ")
phone = input("Enter your phone number: ")
linkedin = input("Enter your LinkedIn profile: ")

# Get user skills and skill ratings
skills = []
skill_ratings = []

while True:
    skill = input("Enter a skill (or 'done' to finish): ")
    if skill.lower() == 'done':
        break
    rating = input(f"Rate your proficiency in {skill} from 1 to 10: ")
    skills.append(skill)
    skill_ratings.append(int(rating))

# Convert user skills and ratings into NumPy arrays
user_skills_vectors = vectorizer.transform(skills).toarray()

# Stack the vectors together
user_vectors = np.hstack([user_skills_vectors, np.array([skill_ratings]).T])

# Predict the job title for the user
predicted_job_title = classifier.predict(user_vectors)[0]

# Generate a resume content
resume_content = f"""
**Name:** {name}
**Email:** {email}
**Phone:** {phone}
**LinkedIn:** {linkedin}

**Summary**

Machine learning engineer with 5+ years of experience in developing and deploying machine learning models for real-world problems. Proven ability to apply machine learning techniques to solve a variety of business challenges, including fraud detection, customer segmentation, and product recommendation.

**Skills and Ratings**

"""
for skill, rating in zip(skills, skill_ratings):
    resume_content += f"* {skill}: {rating}/10\n"

resume_content += f"""
**Predicted Job Title**

{predicted_job_title}
"""

# Create a BytesIO object to store the PDF content
pdf_buffer = BytesIO()

# Create a PDF canvas
pdf_canvas = canvas.Canvas(pdf_buffer, pagesize=(8.5 * 72, 11 * 72))

# Set font and size
pdf_canvas.setFont("Helvetica", 12)

# Add the resume content to the PDF
text_object = pdf_canvas.beginText(100, 800)
text_object.setTextOrigin(100, 800)
text_object.setFont("Helvetica", 12)
text_object.setTextOrigin(100, 800)

# Split the content into lines and add to the PDF
for line in resume_content.split('\n'):
    text_object.textLine(line)

# Draw the text on the canvas
pdf_canvas.drawText(text_object)

# Save the PDF content
pdf_canvas.save()

# Move the buffer position to the beginning
pdf_buffer.seek(0)

# Save the PDF to a file
with open('my_pdf.pdf', 'wb') as pdf_file:
    pdf_file.write(pdf_buffer.read())

ModuleNotFoundError: No module named 'reportlab'